In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
17327,2024-03-23,Brasil Nbb,17:00,Pinheiros,Corinthians Paulista,1.76,1.97,156.5,1.87,1.79,-2.5,2.03,2.37,WGSO6WJ3,0.568182,0.507614,0.534759,0.558659,0.075796,0.4,0.6,NaN,NaN,169.890,58.051012,0.341698,152.200,70.676102,0.464363,155.840,162.806,142.71,248.40,0.0,0.0,0.0,0.0,0.079621,0.030912,0.109280,-1.56,-0.312,-2.435897,0.557831,0.4,-0.157831,2.25,0.450,2.155556,0.493680,0.5,0.006320
17328,2024-03-23,Eua Nba,20:00,Orlando Magic,Sacramento Kings,1.54,2.26,215.5,1.89,1.97,-3.5,1.89,2.65,lnxWg1oQ,0.649351,0.442478,0.529101,0.507614,0.091829,0.8,0.8,NaN,NaN,186.982,30.964819,0.165603,213.212,58.876353,0.276140,183.562,296.340,227.48,286.00,0.0,0.0,0.0,0.0,0.267956,0.029310,0.236741,1.18,0.236,2.288136,0.542133,0.6,0.057867,-0.05,-0.010,-126.000000,0.466745,0.5,0.033255
17329,2024-03-23,Itália Liga A,15:00,Pistoia,Treviso,1.53,2.53,159.5,1.84,1.92,-4.5,1.90,4.20,ObajDoGA,0.653595,0.395257,0.543478,0.520833,0.048852,0.0,0.4,NaN,NaN,263.532,129.580397,0.491706,214.324,62.363913,0.290980,183.908,237.280,171.36,246.50,0.0,0.0,0.0,0.0,0.348328,0.030090,0.533228,0.52,0.104,5.096154,0.451258,0.6,0.148742,3.33,0.666,2.297297,0.000000,0.0,0.000000
17330,2024-03-23,Itália Liga A,16:30,Trento,Sassari,1.71,2.15,160.5,1.83,1.93,-3.5,2.00,2.75,Uy0nER04,0.584795,0.465116,0.546448,0.518135,0.049912,0.8,0.0,NaN,NaN,161.778,49.997936,0.309053,185.754,79.653025,0.428809,149.136,192.700,116.44,187.86,0.0,0.0,0.0,0.0,0.161206,0.037612,0.223297,-2.08,-0.416,-1.706731,0.000000,0.0,0.000000,-3.33,-0.666,-1.726727,0.000000,0.0,0.000000
17331,2024-03-23,Alemanha Bbl,16:00,Crailsheim Merlins,Gottingen,1.83,1.98,176.5,1.80,1.94,-1.5,1.94,2.11,Kd5eK9xI,0.546448,0.505051,0.555556,0.515464,0.051499,0.4,0.8,NaN,NaN,524.342,573.148561,1.093082,351.168,261.524750,0.744728,226.952,380.436,333.75,799.00,1.0,1.0,0.0,0.0,0.055678,0.052938,0.059362,-3.27,-0.654,-1.269113,0.000000,0.0,0.000000,-2.64,-0.528,-1.856061,0.000000,0.0,0.000000
17332,2024-03-23,Alemanha Pro A,15:00,Bochum,Dresden,1.95,1.77,176.5,1.84,1.86,1.5,1.81,1.64,KYI4EdL2,0.512821,0.564972,0.543478,0.537634,0.077792,0.2,0.6,NaN,NaN,176.172,44.433802,0.252218,148.826,33.432144,0.224639,166.648,172.800,197.96,185.31,0.0,0.0,0.0,0.0,0.068430,0.007644,0.069686,0.14,0.028,33.928571,0.500644,0.5,-0.000644,-3.73,-0.746,-1.032172,0.000000,0.0,0.000000
17333,2024-03-23,Argentina Liga A,20:00,Platense,Boca Juniors,2.07,1.70,154.5,1.86,1.80,1.5,1.92,1.56,lEGTtld7,0.483092,0.588235,0.537634,0.555556,0.071327,0.8,0.6,NaN,NaN,184.366,111.903565,0.606964,111.152,10.591285,0.095287,111.362,131.952,111.35,143.56,0.0,0.0,0.0,0.0,0.138795,0.023184,0.146298,0.44,0.088,12.159091,0.000000,0.0,0.000000,-0.48,-0.096,-7.291667,0.000000,0.0,0.000000
17334,2024-03-23,Bulgária Nbl,13:00,Yambol,Levski,1.42,2.72,155.5,1.83,1.87,2.5,2.01,1.94,ImgBM298,0.704225,0.367647,0.546448,0.534759,0.071872,0.6,0.4,NaN,NaN,273.050,86.192676,0.315666,289.338,112.829917,0.389959,295.378,421.416,377.25,385.70,1.0,0.0,0.0,0.0,0.444077,0.015289,0.025062,-1.98,-0.396,-1.060606,0.000000,0.0,0.000000,-5.00,-1.000,-1.720000,0.000000,0.0,0.000000
17335,2024-03-23,Eslováquia Extraliga,14:00,Handlova,BC Komarno,2.03,1.71,161.5,1.85,1.83,1.5,1.90,1.57,pjyStHEt,0.492611,0.584795,0.540541,0.546448,0.077406,0.8,0.6,NaN,NaN,155.060,6.966061,0.044925,126.630,21.487562,0.169688,128.930,144.764,149.76,95.94,0.0,0.0,0.0,0.0,0.121002,0.007686,0.134493,0.30,0.060,17.166667,0.000000,0.0,0.000000

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,17:00,Brasil Nbb,Pinheiros,Corinthians Paulista,156.5,1.87,1.0000,Over
1,20:00,Eua Nba,Orlando Magic,Sacramento Kings,215.5,1.89,1.0000,Over
2,15:00,Itália Liga A,Pistoia,Treviso,159.5,1.84,1.0000,Over
3,16:30,Itália Liga A,Trento,Sassari,160.5,1.83,1.0000,Over
4,16:00,Alemanha Bbl,Crailsheim Merlins,Gottingen,176.5,1.80,0.9991,Over
5,13:00,Bulgária Nbl,Yambol,Levski,155.5,1.83,0.9919,Over
6,13:45,Eua Ncaa,Arizona,Dayton,148.5,1.80,1.0000,Over
7,18:30,Eua Ncaa,North Carolina,Michigan State,139.5,1.80,1.0000,Over
8,19:10,Eua Ncaa,Iowa State,Washington State,128.5,1.82,1.0000,Over
9,21:40,Eua Ncaa,Illinois,Duquesne,147.5,1.80,1.0000,Over
